# Data Wrangling January 2020 #
## Lanie Preston & Floor Kouwenberg ##

Lanie Preston (VU Student Nr: 2629403)

Floor Kouwenberg (VU ID:

In [1]:
#read in libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup


## Data on USA Dairy Production per Year and Quarter ## 

In [81]:
us_dairy_production = pd.read_excel('annualmilkprodfactors.xlsx', index_col=0)
us_dairy_production = us_dairy_production.dropna(axis=0)
#us_dairy_production = us_dairy_production.reset_index
milk_production =  us_dairy_production[['Year', 'Milk production']]

KeyError: "['Year'] not in index"

## Data on Party Divisions per Year ##

In [45]:
#seats per party per year 
senate_division = pd.read_excel('senate_division.xlsx')
senate_division

,Congress,Years,Total,Democrats,Republicans,Others,Vacancies,HOR: Total,HOR: Democrats,HOR: Republicans,HOR: Other,HOR: Vacancies
0,105,1997-1999,100,45,55,NaN,NaN,435,206,227,2.0,NaN
1,106,1999–2001,100,45,55,NaN,NaN,435,211,223,1.0,NaN
2,107,2001–2003,100,50,50,NaN,NaN,435,212,221,2.0,NaN
3,108,2003–2005,100,48,51,1.0,NaN,435,205,229,1.0,NaN
4,109,2005–2007,100,44,55,1.0,NaN,435,202,232,1.0,NaN
5,110,2007–2009,100,49,49,2.0,NaN,435,233,202,NaN,NaN
6,111,2009–2011,100,57,41,1.0,1.0,435,257,178,NaN,NaN
7,112,2011–2013,100,51,47,2.0,NaN,435,193,242,NaN,NaN
8,113,2013–2015,100,53,45,2.0,NaN,435,201,234,NaN,NaN
9,114,2015–2017,100,44,54,2.0,NaN,435,188,247,NaN,NaN


## Data on Dollar Amounts of Subsudies paid to Dairy Farmers by the US Government ##

In [103]:
#government payouts by program
#read in separate excel files 
payouts2010 = pd.read_excel('GP_PaymentsUSA2010.xlsx')
payouts2000 = pd.read_excel('GP_PaymentsUSA2000.xlsx')
payouts1990 = pd.read_excel('GP_PaymentsUSA1990.xlsx')

#give all dfs sensible column names 
def clean_df(df):
    if 'Unnamed: 4' in list(df):
        del df['Unnamed: 4']
    df.rename(columns= {'Unnamed: 0': 'Program'}, inplace=True)
    df.rename(columns= {'United States': 'Program'}, inplace=True)
    return df

#clean all dataframes for merging 
payouts1990 = clean_df(payouts1990)
payouts2000 = clean_df(payouts2000)
payouts2010 = clean_df(payouts2010)

payouts_early = pd.merge(payouts1990, payouts2000, how='outer')
payouts_early = payouts_early.drop([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997], axis=1)
all_payouts = pd.merge(payouts_early, payouts2010, how='outer')
all_payouts = all_payouts.set_index('Program')

search_values = ['dairy','milk']
dairy_payouts = all_payouts[all_payouts.index.str.contains('|'.join(search_values),  na=False, case=False)]

,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Program,,,,,,,,,,,,,,,,,,,,,
Milk income loss payments,NaN,NaN,NaN,NaN,859642,913317,205677,9587,431231,73902,...,51660.0,-100.0,446572.0,231704.0,-129.0,-40.0,-64.0,-43.0,-20.0,NaN
Dairy Margin Coverage Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,686.0,10394.0,19.0,250013.0,214000.0


In [ ]:
#lobbies per year per party - how do we read this in?
lobbies = pd.read_excel('AnnualLobyingOnDairy.xlsx', index_col=0)
#reset index so years is a column
lobbies.reset_index(level=0, inplace=True)
lobbies_final = lobbies.pivot_table('Dollar', columns='Year')
pd.concat([dairy_payouts, lobbies_final], axis=1, sort=False)